# **Value Iteration & Policy Iteration**

In [234]:
!pip install gymnasium

Defaulting to user installation because normal site-packages is not writeable


In [235]:
import gymnasium as gym
import numpy as np
import time
from IPython import display

In [236]:
envs = ['FrozenLake-v1', 'FrozenLake8x8-v1', 'Taxi-v3']

In [237]:
def play(env, policy, render=False):
    state, _ = env.reset()
    total_reward = 0
    steps = 0
    done = False
    while not done:
        action = policy[state]
        next_state, reward, done, info, _ = env.step(action)
        total_reward += reward
        steps += 1
        if render:
            print(env.render())
            time.sleep(0.5)
            if not done:
                display.clear_output(wait=True)
        state = next_state

    return (total_reward, steps)

In [238]:
def play_multiple_times(env, policy, max_episodes):
    success = 0
    list_of_steps = []
    for i in range(max_episodes):
        total_reward, steps = play(env, policy)

        if total_reward > 0:
            success += 1
            list_of_steps.append(steps)

    print(f'Number of successes: {success}/{max_episodes}')
    print(f'Average number of steps: {np.mean(list_of_steps)}')

## Policy Iteration

### *Policy Evaluation Function*

In [239]:
def policy_evaluation(env, policy, gamma=0.9, theta=1e-13):
    
    v = np.zeros(env.observation_space.n)
    
    while True:
        delta = 0
        for s in range(env.observation_space.n):
            v_temp = v[s]
            action = policy[s]
            v_new = 0
            for prob, next_state, reward, _ in env.P[s][action]:
                v_new += prob * (reward + gamma * v[next_state])
            v[s] = v_new
            delta = max(delta, abs(v[s] - v_temp))
        if delta < theta:
            break

    return v

### *Policy Iteration Function*

In [240]:
def policy_iteration(env, gamma=0.9, theta=1e-13):
    
    policy = np.zeros(env.observation_space.n, dtype=int)
    v = np.zeros(env.observation_space.n)
    i = 0
    
    while True:
        i += 1
        v = policy_evaluation(env, policy, gamma, theta)
        policy_stable = True
        for s in range(env.observation_space.n):
            old_action = policy[s]
            action_values = np.zeros(env.action_space.n)
            for a in range(env.action_space.n):
                for prob, next_state, reward, _ in env.P[s][a]:
                    action_values[a] += prob * (reward + gamma * v[next_state])
            policy[s] = np.argmax(action_values)
            if policy[s] != old_action:
                policy_stable = False
        if policy_stable:
            print(f'Converged at {i}-th iteration.')
            break

    return policy, v

In [310]:
time_taken_1 = []
for i in range(len(envs)):
    time_taken_1.append(0)

In [319]:
optimal_v_values = []
optimal_policies = []
for i, env_name in enumerate(envs):
    env = gym.make(env_name, render_mode="ansi")
    start = time.time()
    policy, v = policy_iteration(env)
    optimal_policies.append(policy)
    optimal_v_values.append(v)
    end = time.time()
    time_taken_1[i] += (end - start)
    print(f"{env_name}: {end - start}s")

Converged at 6-th iteration.
FrozenLake-v1: 0.07380294799804688s
Converged at 10-th iteration.
FrozenLake8x8-v1: 0.692147970199585s
Converged at 18-th iteration.
Taxi-v3: 17.65279221534729s


In [332]:
#Trung bình thời gian chạy của 3 toy games sau 5 lần
for i, env_name in enumerate(envs):
    print(f"{env_name}: {time_taken_1[i]/5}s")

FrozenLake-v1: 0.07759213447570801s
FrozenLake8x8-v1: 0.7214705467224121s
Taxi-v3: 17.63923931121826s


In [259]:
for i in optimal_v_values:
    print(i)

[0.06888615 0.06141054 0.07440682 0.05580409 0.09185022 0.
 0.11220663 0.         0.14543286 0.2474946  0.29961593 0.
 0.         0.3799342  0.63901926 0.        ]
[0.00641104 0.00854808 0.01230044 0.01778942 0.02508214 0.03247089
 0.03957134 0.04297844 0.00602405 0.00764512 0.01091162 0.01642654
 0.02605411 0.03619409 0.0493547  0.05730461 0.00509024 0.0058532
 0.00677534 0.         0.02557084 0.03882139 0.06763973 0.08435607
 0.0042256  0.00476954 0.00581968 0.0078541  0.02036065 0.
 0.09175501 0.12919111 0.00318093 0.00319659 0.00270488 0.
 0.0344439  0.06195145 0.10901921 0.20969093 0.00186915 0.
 0.         0.01085079 0.03250092 0.06304172 0.         0.36008773
 0.00118046 0.         0.00137717 0.00366839 0.         0.11568671
 0.         0.63051379 0.00088531 0.00077462 0.00092218 0.
 0.13824885 0.32258065 0.61443932 0.        ]
[ 89.47323891  32.81971401  55.26423891  37.57755845   8.43222921
  32.81971401   8.43222921  15.28447953  32.81971401  18.09386122
  55.26423891  21.215

In [260]:
for i in optimal_policies:
    print(i)

[0 3 0 3 0 0 0 0 3 1 0 0 0 2 1 0]
[3 2 2 2 2 2 2 2 3 3 3 3 2 2 2 1 3 3 0 0 2 3 2 1 3 3 3 1 0 0 2 1 3 3 0 0 2
 1 3 2 0 0 0 1 3 0 0 2 0 0 1 0 0 0 0 2 0 1 0 0 1 1 1 0]
[4 4 4 4 0 0 0 0 0 0 0 0 0 0 0 0 5 0 0 0 3 3 3 3 0 0 0 0 0 0 0 0 0 0 0 0 3
 0 0 0 0 0 0 0 2 2 2 2 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 2 2 2 2 0 0 0 0 0 0
 0 0 0 2 0 0 0 0 0 0 4 4 4 4 0 0 0 0 0 0 0 0 0 5 0 0 1 1 1 1 0 0 0 0 0 0 0
 0 0 0 0 0 1 0 0 0 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 1 1 1
 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 1 1 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1
 1 1 1 0 0 0 0 0 0 0 0 0 1 0 0 1 1 1 1 2 2 2 2 0 0 0 0 2 2 2 2 1 2 0 2 1 1
 1 1 2 2 2 2 3 3 3 3 2 2 2 2 1 2 3 2 3 3 3 3 1 1 1 1 3 3 3 3 2 2 2 2 3 1 3
 2 3 3 3 3 1 1 1 1 3 3 3 3 0 0 0 0 3 1 3 0 3 3 3 3 1 1 1 1 3 3 3 3 0 0 0 0
 3 1 3 0 1 1 1 1 1 1 1 1 0 0 0 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 0 0 0 0 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 1 1 1 0 1 1 1 1 1 1 1
 1 4 4 4 4

In [245]:
for i, env_name in enumerate(envs):
    env = gym.make(env_name, render_mode="ansi")
    play(env, optimal_policies[i], True)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)



In [265]:
for i, env_name in enumerate(envs):
    env = gym.make(env_name, render_mode="ansi")
    play_multiple_times(env, optimal_policies[i], 1000)

Number of successes: 778/1000
Average number of steps: 41.80848329048843
Number of successes: 740/1000
Average number of steps: 73.72972972972973
Number of successes: 1000/1000
Average number of steps: 13.011


## Value Iteration

### *Policy Evaluation Function*

In [ ]:
def policy_evaluation(env, policy, max_iters=500, gamma=0.9):
    # Initialize the values of all states to be 0
    v_values = np.zeros(env.observation_space.n)

    for i in range(max_iters):
        prev_v_values = np.copy(v_values)

        # Update the value of each state
        for state in range(env.observation_space.n):
            action = policy[state]

            # Compute the q-value of the action
            q_value = 0
            for prob, next_state, reward, done in env.P[state][action]:
                q_value += prob * (reward + gamma * prev_v_values[next_state])

            v_values[state] = q_value # update v-value

        # Check convergence
        if np.all(np.isclose(v_values, prev_v_values)):
            print(f'Converged at {i}-th iteration.')
            break

    return v_values

### *Value Iteration Function*

In [296]:
def value_iteration(env, max_iters=500, gamma=0.9):
    # initialize
    v_values = np.zeros(env.observation_space.n)

    for i in range(max_iters):
        prev_v_values = np.copy(v_values)

        # update the v-value for each state
        for state in range(env.observation_space.n):
            q_values = []

            # compute the q-value for each action that we can perform at the state
            for action in range(env.action_space.n):
                q_value = 0
                # loop through each possible outcome
                for prob, next_state, reward, done in env.P[state][action]:
                    q_value += prob * (reward + gamma * prev_v_values[next_state])

                q_values.append(q_value)

            # select the max q-values
            best_action = np.argmax(q_values)
            v_values[state] = q_values[best_action]

        # check convergence
        if np.all(np.isclose(v_values, prev_v_values)):
            print(f'Converged at {i}-th iteration.')
            break

    return v_values

In [341]:
time_taken_2 = []
for i in range(len(envs)):
    time_taken_2.append(0)

In [350]:
optimal_v_values = []
for i, env_name in enumerate(envs):
    env = gym.make(env_name, render_mode="ansi")
    start = time.time()
    v = value_iteration(env)
    optimal_v_values.append(v)
    end = time.time()
    time_taken_2[i] += (end - start)
    print(f"{env_name}: {end - start}s")

Converged at 79-th iteration.
FrozenLake-v1: 0.04886913299560547s
Converged at 117-th iteration.
FrozenLake8x8-v1: 0.25531625747680664s
Converged at 116-th iteration.
Taxi-v3: 2.639939785003662s


In [352]:
#Trung bình thời gian chạy của 3 toy games sau 5 lần
for i, env_name in enumerate(envs):
    print(f"{env_name}: {time_taken_2[i]/5}s")

FrozenLake-v1: 0.0486696720123291s
FrozenLake8x8-v1: 0.30558276176452637s
Taxi-v3: 2.655098867416382s


In [254]:
for i in optimal_v_values:
    print(i)

[0.06888615 0.06141054 0.07440682 0.05580409 0.09185022 0.
 0.11220663 0.         0.14543286 0.2474946  0.29961593 0.
 0.         0.3799342  0.63901926 0.        ]
[0.00641104 0.00854808 0.01230044 0.01778942 0.02508214 0.03247089
 0.03957134 0.04297844 0.00602405 0.00764512 0.01091162 0.01642654
 0.02605411 0.03619409 0.0493547  0.05730461 0.00509024 0.0058532
 0.00677534 0.         0.02557084 0.03882139 0.06763973 0.08435607
 0.0042256  0.00476954 0.00581968 0.0078541  0.02036065 0.
 0.09175501 0.12919111 0.00318093 0.00319659 0.00270488 0.
 0.0344439  0.06195145 0.10901921 0.20969093 0.00186915 0.
 0.         0.01085079 0.03250092 0.06304172 0.         0.36008773
 0.00118046 0.         0.00137717 0.00366839 0.         0.11568671
 0.         0.63051379 0.00088531 0.00077462 0.00092218 0.
 0.13824885 0.32258065 0.61443932 0.        ]
[ 89.47323891  32.81971401  55.26423891  37.57755845   8.43222921
  32.81971401   8.43222921  15.28447953  32.81971401  18.09386122
  55.26423891  21.215

### *Policy Extraction Function*

In [261]:
def policy_extraction(env, v_values, gamma=0.9):
    # initialize
    policy = np.zeros(env.observation_space.n, dtype=np.int32)

    # loop through each state in the environment
    for state in range(env.observation_space.n):
        q_values = []
        # loop through each action
        for action in range(env.action_space.n):
            q_value = 0
            # loop each possible outcome
            for prob, next_state, reward, done in env.P[state][action]:
                q_value += prob * (reward + gamma * v_values[next_state])

            q_values.append(q_value)

        # select the best action
        best_action = np.argmax(q_values)
        policy[state] = best_action

    return policy

In [262]:
optimal_policies = []
for i, env_name in enumerate(envs):
    env = gym.make(env_name, render_mode="ansi")
    policy = policy_extraction(env, optimal_v_values[i])
    optimal_policies.append(policy)

C:\Users\ZVC\AppData\Roaming\Python\Python312\site-packages\gymnasium\core.py:311: UserWarning: WARN: env.P to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.P` for environment variables or `env.get_wrapper_attr('P')` that will search the reminding wrappers.
  logger.warn(


In [264]:
for i in optimal_policies:
    print(i)

[0 3 0 3 0 0 0 0 3 1 0 0 0 2 1 0]
[3 2 2 2 2 2 2 2 3 3 3 3 2 2 2 1 3 3 0 0 2 3 2 1 3 3 3 1 0 0 2 1 3 3 0 0 2
 1 3 2 0 0 0 1 3 0 0 2 0 0 1 0 0 0 0 2 0 1 0 0 1 1 1 0]
[4 4 4 4 0 0 0 0 0 0 0 0 0 0 0 0 5 0 0 0 3 3 3 3 0 0 0 0 0 0 0 0 0 0 0 0 3
 0 0 0 0 0 0 0 2 2 2 2 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 2 2 2 2 0 0 0 0 0 0
 0 0 0 2 0 0 0 0 0 0 4 4 4 4 0 0 0 0 0 0 0 0 0 5 0 0 1 1 1 1 0 0 0 0 0 0 0
 0 0 0 0 0 1 0 0 0 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 1 1 1
 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 1 1 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1
 1 1 1 0 0 0 0 0 0 0 0 0 1 0 0 1 1 1 1 2 2 2 2 0 0 0 0 2 2 2 2 1 2 0 2 1 1
 1 1 2 2 2 2 3 3 3 3 2 2 2 2 1 2 3 2 3 3 3 3 1 1 1 1 3 3 3 3 2 2 2 2 3 1 3
 2 3 3 3 3 1 1 1 1 3 3 3 3 0 0 0 0 3 1 3 0 3 3 3 3 1 1 1 1 3 3 3 3 0 0 0 0
 3 1 3 0 1 1 1 1 1 1 1 1 0 0 0 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 0 0 0 0 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 1 1 1 0 1 1 1 1 1 1 1
 1 4 4 4 4

In [257]:
for i, env_name in enumerate(envs):
    env = gym.make(env_name, render_mode="ansi")
    play(env, optimal_policies[i], True)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)



In [266]:
for i, env_name in enumerate(envs):
    env = gym.make(env_name, render_mode="ansi")
    play_multiple_times(env, optimal_policies[i], 1000)

Number of successes: 773/1000
Average number of steps: 43.31953428201811
Number of successes: 729/1000
Average number of steps: 76.34293552812072
Number of successes: 1000/1000
Average number of steps: 12.907


## *Nhận xét:*
- Cả Policy Iteration và Value Iteration đều là những thuật toán để tìm chính sách tối ưu. 
  + **Policy Iteration** sử dụng hai bước lặp lại: (1) *Policy Evaluation* để tính giá trị cho một chính sách cho trước, và (2) *Policy Iteration* để cập nhật chính sách dựa trên giá trị đó. Nó thường hội tụ nhanh hơn **Value Iteration**, nhưng mỗi vòng lặp tốn nhiều thời gian hơn vì phải tính toán giá trị hàm giá trị cho mỗi chính sách.
  + **Value Iteration** tính toán trực tiếp giá trị hàm giá trị tối ưu cho mỗi trạng thái, và từ đó suy ra chính sách tối ưu. Mỗi vòng lặp của nó tốn ít thời gian hơn so với **Policy Iteration**, nhưng có thể cần nhiều vòng lặp hơn để hội tụ.
- So sánh thời gian chạy của 2 thuật toán trong các toy games sau 5 lần: ở cả 3 toy games, thuật toán **Value Iteration** đều cho ra kết quả nhanh hơn so với thuật toán **Policy Iteration** cụ thể là **Value Iteration** (0.05; 0.3; 2.6) và **Policy Iteration** (0.07; 0.7; 17.7) tương ứng với các toy games (*FrozenLake-v1*; *FrozenLake8x8-v1*; *Taxi-v3*).